setteing backend to tf

In [3]:
import keras

Using Theano backend.


AttributeError: 'module' object has no attribute 'tests'

In [19]:
%matplotlib inline
import utils_tf; reload(utils_tf)
from utils_tf import *
from __future__ import division, print_function

In [20]:
import tensorflow as tf
from keras import backend as K
sess = tf.Session()
K.set_session(sess)

In [2]:
keras.backend.backend()
keras.backend.image_dim_ordering()

Using Theano backend.


AttributeError: 'module' object has no attribute 'tests'

In [22]:
def slice_batch(x, n_gpus, part):
    sh = K.shape(x)
    L = sh[0] / n_gpus
    if part == n_gpus - 1:
        return x[part*L:]
    return x[part*L:(part+1)*L]

In [23]:
def to_multi_gpu(model, n_gpus=2):
    with tf.device('/cpu:0'):
        x = Input(model.input_shape[1:])#, name=model.input_names[0])
    towers = []
    for g in range(n_gpus):
        with tf.device('/gpu:' + str(g)):
            slice_g = Lambda(slice_batch, lambda shape: shape,arguments={'n_gpus':n_gpus, 'part':g})(x)
            towers.append(model(slice_g))
        with tf.device('/cpu:0'):
            merged = merge(towers, mode='concat', concat_axis=0)
    return Model(input=[x], output=merged)

### tensorflow model - sample

In [24]:

img = tf.placeholder(tf.float32, shape=(None, 784))

In [25]:
from keras.layers import Dense

# Keras layers can be called on TensorFlow tensors:
x = Dense(128, activation='relu')(img)  # fully-connected layer with 128 units and ReLU activation
x = Dense(128, activation='relu')(x)
preds = Dense(10, activation='softmax')(x)  # output layer with 10 units and a softmax activation

In [26]:
labels = tf.placeholder(tf.float32, shape=(None, 10))

from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

### trying to use my things on Tensorflow

In [27]:
path = "/home/ubuntu/Notebooks/Kaggle_Fisherman/"
batch_size=64

In [28]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(path)

#seems script didnt get the test files... find out why later

Found 3422 images belonging to 8 classes.
Found 355 images belonging to 8 classes.
Found 3422 images belonging to 8 classes.
Found 355 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [30]:
tf_shape=(224,224,3)
th_shape=(3,224,224)
if keras.backend.image_dim_ordering()=='th':
    in_shape=th_ishape
else:
    in_shape=tf_shape
#tensorflow: 70 secs per epoch
model = Sequential([
        BatchNormalization(axis=1, input_shape=in_shape),
        Flatten(),
        Dense(8, activation='softmax')
    ])
#model=to_multi_gpu(model,2)
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
 256/3422 [=>............................] - ETA: 73s - loss: 2.4171 - acc: 0.1680 

KeyboardInterrupt: 

84s - loss: 1.4813 - acc: 0.5161 - val_loss: 1.5857 - val_acc: 0.56
74s - loss: 0.8099 - acc: 0.7499 - val_loss: 0.6555 - val_acc: 0.78           
                